<center>

<h1> Programming with Lists </h1>
<h2> CS3100 Monsoon 2020 </h2>
</center>

## Review

### Previously

* Solving a logic puzzle

### This lecture

* Programming with Lists
* And many concepts in the process
  + Arithmetic
  + Last call optimisation
  + Backtracking & Choice points.

## Trouble with Zebras

* Our zebra puzzle only works with 5 houses.
  + Problem is the 5-tuple `street(H1,H2,H3,H4,H5)`.
* Need to rewrite the entire program if more houses were added.
* What we really need is **lists**.

## Support for lists in Prolog

* Notated with square brackets `[1,2,3,4]`.
* The empty list is `[]`.
* List pattern matching is `[H | T]`, where `H` is a list element and `T` is a list.
  + Can also match `[1,2,3 | T]`.


## Quiz

Which of these unify?

1. `[1,2] = [H | [2]]`
2. `[1,2] = [1 | 2]`
3. `[1,2] = [X,Y | []]`
4. `[1,2] = [ _ | [1,2]]`

## Quiz

Which of these unify?

1. `[1,2] = [H | [2]]` **yes - H=1**
2. `[1,2] = [1 | 2]` **no - RHS is not a list**
3. `[1,2] = [X,Y | []]` **yes - X=1,Y=2**
4. `[1,2] = [ _ | [1,2]]` **no - RHS unifies with a list `[X,1,2]`**

## Finding the last element of the list

In [2]:
last([H],H).
last([_ | T], V) :- last(T, V).

In [3]:
?- last([1,2],X).

X = 2 .

## Tracing the example by hand

    last([1,2],X).

## Quiz
```prolog
last([H],H).
last([_ | T], V) :- last(T, V).
```
What happens if I ask for `last([],X)`?

1. pattern match exception
2. Prolog says false.
3. Prolog says true, X = [].
4. Prolog says true, X = ???.

## Quiz
```prolog
last([H],H).
last([_ | T], V) :- last(T, V).
```
What happens if I ask for `last([],X)`?

1. pattern match exception
2. Prolog says false. **✓**
3. Prolog says true, X = [].
4. Prolog says true, X = ???.

## Tracing the example in SWI-Prolog

    last ([1,2],X)
    
Show what happens with `;` after the first result.

## Arithmetic

* How do we compute the **length** of the list?
  + We need support for arithmetic. 
* Arithmetic is quite natural in imperative and functional paradigms.
  + Since **computation is deduction** in logic programming, arithmetic is quite special.

## Arithmetic equality != Unification

`=` operator is used up by unification.

In [4]:
?- A = 1+2.

A = +(1, 2) .

In [5]:
?- 1+2 = 3.

false.

In [6]:
?- A = money+power.

A = +(money, power) .

## Use the `is` operator

The “is” operator tells prolog to evaluate the righthand expression numerically and unify with the left.

In [7]:
?- X is 1, A is X+2.

X = 1, A = 3 .

In [8]:
?- A is money+power.

: 

## Restriction on `is` operator

The RHS must be a ground term (no variables). Prolog doesn't do algebra.

In [9]:
?- A is B+2.

: 

In [13]:
?- 3 is B+2.

: 

## Quiz

What is the result of `A is *(3,+(1,2))?`

1. Error
2. 9
3. 8
4. 6

## Quiz

What is the result of `A is *(3,+(1,2))?`

1. Error
2. 9 **✓**
3. 8
4. 6

## Arithmetic

There is support for `+,*, /, <, =<, >, >=`,etc.

In [16]:
?- A is 20 / 20.

A = 1 .

## List Sum

Compute the sum of the list. This is the example we saw in the first Prolog lecture. 

In [17]:
sum([],0).
sum([H | T], N) :- sum(T,M), N is M+H.

Added 2 clauses(s).

In [18]:
?- sum([1,2,3],X).

X = 6 .

In [19]:
?- sum(X,3) {1}.

: 

Trace `?- sum(X,3)` by hand and in `swipl`.

## Length of list

In [1]:
len([],0).
len([_ | T], N) :- len(T,M), N is M+1.

Added 2 clauses(s).

In [2]:
?- len([1,2,3],X).

X = 3 .

## Last call optimisation

* `len` uses O(N) stack space.

## Trace `len` by hand
    
    ?- len([1,2],X)

## Tail recursive length

In [3]:
len2([],Acc,Acc).
len2([H|T],Acc,N) :- M is Acc+1, len2(T,M,N).

Added 2 clauses(s).

In [4]:
?- len2([1,2],0,X).

X = 2 .

## Trace `len2` by hand.

    ?- len2([1,2],0,X).

## Predicate Overloading

In [5]:
len2(L,X) :- len2(L,0,X).

Added 1 clauses(s).

In [6]:
?- len2([1,2,3],X).

X = 3 .

## Last Call Optimisation

* This technique is applied by the prolog interpreter
* The last clause of the rule is executed as a branch and not a call
* We can only do this if the rule is **determinate** up to that point
  + No further choices for the rule
  + Relates to **choice points** (to be seen).

## List append

In [7]:
append([],Q,Q).
append([H | P], Q, [H | R]) :- append(P, Q, R).

Added 2 clauses(s).

In [8]:
?- append([1,2],[3,4],X).

X = [ 1, 2, 3, 4 ] .

In [9]:
?- append([1,2],X,[1,2,3,4]).

X = [ 3, 4 ] .

In [10]:
?- append(X,Y,[1,2,3,4]).

X = [  ], Y = [ 1, 2, 3, 4 ] ;
X = [ 1 ], Y = [ 2, 3, 4 ] ;
X = [ 1, 2 ], Y = [ 3, 4 ] ;
X = [ 1, 2, 3 ], Y = [ 4 ] ;
X = [ 1, 2, 3, 4 ], Y = [  ] .

## Prefix and Suffix

Prefix and Suffix of list can be defined using append.

In [11]:
prefix(X,Z) :- append(X,Y,Z).
suffix(Y,Z) :- append(X,Y,Z).

Added 2 clauses(s).

## Prefix and Suffix

In [12]:
?- prefix(X,[1,2,3]).

X = [  ] ;
X = [ 1 ] ;
X = [ 1, 2 ] ;
X = [ 1, 2, 3 ] .

In [13]:
?- suffix(X,[1,2,3]).

X = [ 1, 2, 3 ] ;
X = [ 2, 3 ] ;
X = [ 3 ] ;
X = [  ] .

## Backtracking & Choice Points

* The way prolog fetches multiple results for the given query is through **Backtracking**. 
* **Choice points** are locations in the search where we could take another option.
  + If there are no choice points left then Prolog doesn't offer the user any more answers

## Trace `prefix` by hand

    ?- prefix(X,[1,2]).

## Quiz
```prolog
len([],0).
len([_ | T], N) :- len(T,M), N is M+1.
```
What is the first result of query `len(A,2)`?

1. Error due uninstantiated arithmetic expression.
2. A = [\_,\_]
3. Query runs forever
4. Error due to invalid arguments

## Quiz
```prolog
len([],0).
len([_ | T], N) :- len(T,M), N is M+1.
```
What is the first result of query `len(A,2)`?

1. Error due uninstantiated arithmetic expression.
2. A = [\_,\_] **✓**
3. Query runs forever
4. Error due to invalid arguments

## Trace `len` by hand

    ?- len(A,2)

## Quiz
```prolog
len([],0).
len([_ | T], N) :- len(T,M), N is M+1.
```
What is the second result of query `len(A,2)`?

1. Error due uninstantiated arithmetic expression.
2. A = [\_,\_] 
3. Query runs forever
4. Error due to invalid arguments

## Quiz
```prolog
len([],0).
len([_ | T], N) :- len(T,M), N is M+1.
```
What is the second result of query `len(A,2)`?

1. Error due uninstantiated arithmetic expression.
2. A = [\_,\_] 
3. Query runs forever **✓**
4. Error due to invalid arguments

## Trace `len` by hand

    ?- len(A,2)

## Limiting the number of results

In [14]:
?- len(A,2) {1}.

A = [ _1604, _1610 ] .

<center>

<h1> Fin. </h1>
</center>